## Data selection and resource access

In this notebook you will:
- filter and collect resources based on properties in the metadata
- carry out simple analysis tasks on custom data collections such as counting files or words

In [67]:
# Make sure lxml is installed
!pip install lxml

# Define some handy defaults
nsmap = {"cmd": "http://www.clarin.eu/cmd/1",
         "cmdp_text": "http://www.clarin.eu/cmd/1/profiles/clarin.eu:cr1:p_1633000337997"}

# Retrieving metadata files

In [68]:
import requests
from io import BytesIO
from zipfile import ZipFile

'''
    This function gets and unpacks a metadata archive
'''
def unpack_metadata(set_id, target_dir):
    # construct the address of the .zip file with the metadata for one set
    md_zip_url = f'https://europeana-oai.clarin.eu/metadata/fulltext-aggregation/{set_id}.zip'
    
    # retrieve the .zip file
    print(f'Retrieving {md_zip_url}')
    resp = requests.get(md_zip_url)
    zipfile = ZipFile(BytesIO(resp.content))
    
    print(f'Extracting content in {target_dir}')
    zipfile.extractall(path=target_dir)
    print('Done')

In [69]:
from os.path import expanduser

'''
    We now call the function for the set of Polish newspapers and extract them in a temporary directory so that we can use it.
'''

set_id = '9200357' # Poland
metadata_dir = f'{expanduser("~")}/work/temp/metadata/{set_id}'
unpack_metadata(set_id, metadata_dir)

Retrieving https://europeana-oai.clarin.eu/metadata/fulltext-aggregation/9200357.zip
Extracting content in /home/jovyan/work/temp/metadata/9200357
Done


# Data segmentation using metadata properties
Now we have the metadata available, we want to use the properties in the metadata to create useful data segments. For The metadata tells us, for instance, the year of publication of each newspaper. 

Thus, we can narrow down our selection of text files and at the same time, using the resource identifiers that are also found in the metadata, create a 'map' that allows us to look up specific files based on a property we are interested in, in this case year of publication.

In [78]:
from lxml import etree

'''
    This function returns the publication year and resource identifiers for a metadata record.
    If the record is not a text resource record, it returns nothing
'''
def get_year_and_ids(record_file):
    xml_tree = etree.parse(record_file)
    text_resource_root_node = xml_tree.xpath('//cmd:Components/cmdp_text:TextResource', namespaces=nsmap)

    if len(text_resource_root_node) < 1:
        # Metadata in file is not for a text resource...
        return None
    else:
        # Get the temporal coverage information
        year_text_node = xml_tree.xpath(
            '//cmdp_text:TextResource/cmdp_text:TemporalCoverage/cmdp_text:Start/cmdp_text:year/text()', namespaces=nsmap)
        if len(year_text_node) < 1:
            # No publication year found... this is unexpected!
            print(f'Warning: year information not found in {record_file}')
            return None
        else:
            year = year_text_node[0]
            # get the record identifiers
            record_ids_text_nodes = xml_tree.xpath(
                '//cmdp_text:SubresourceDescription/cmdp_text:IdentificationInfo/cmdp_text:identifier/text()', namespaces=nsmap)
            
            # we only want the numeric identifiers...
            record_numeric_ids = [id for id 
                                  in record_ids_text_nodes 
                                  if id.isnumeric()]
            
            # return the information found in the record as a tuple
            return(year, record_numeric_ids)    

In [79]:
'''
    Trying it out on a single file that we know to be there...
'''
sample_file = f'{metadata_dir}/Ilustrowany_Dziennik_Ludowy_1939.xml'
get_year_and_ids(sample_file)

('1939', ['3000095232409', '3000095232408', '3000095232410'])

In [91]:
import os

'''
    Now we apply the function to all metadata files and construct an 'index'
    The index will be a dictionary that has the year as 'key' and a list of 
'''
index = {}
years_of_interest = range(1900, 1920)
for filename in os.listdir(metadata_dir):
    if filename.endswith('.xml'):
        info = get_year_and_ids(f'{metadata_dir}/{filename}')
        # did we in fact get metadata for a text resource?
        if info:
            # break down the info
            (year, ids) = info
            if int(year) in years_of_interest: # int(year) gives us a numeric value needed to check against range
                # add to our index
                if not year in index: # does the index have an entry for the year?
                    # if not, make the entry
                    index[year] = []
                # append the identifiers to the entry
                index[year] += ids

# summarize our index: (year, id count)
[(year, len(index[year]))for year in sorted(index)]

[('1900', 50),
 ('1901', 52),
 ('1902', 53),
 ('1903', 51),
 ('1904', 52),
 ('1906', 52),
 ('1907', 51),
 ('1908', 51),
 ('1910', 39),
 ('1911', 53),
 ('1912', 53),
 ('1913', 51),
 ('1914', 1410),
 ('1915', 1939),
 ('1916', 1764),
 ('1917', 1616),
 ('1918', 2092),
 ('1919', 1713)]